<a href="https://colab.research.google.com/github/DzangaR67/Data-science-project_Dropout-analysis/blob/main/Dropout_risk_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# requirements.txt
"""
pandas==2.0.3
numpy==1.24.3
scikit-learn==1.3.0
matplotlib==3.7.2
seaborn==0.12.2
plotly==5.15.0
streamlit==1.28.0
xgboost==1.7.6
imbalanced-learn==0.10.1
jupyter==1.0.0
joblib==1.3.0
ucimlrepo==0.0.6
"""
# Install all required packages
!pip install ucimlrepo pandas numpy scikit-learn matplotlib seaborn plotly xgboost imbalanced-learn joblib

print(" All packages installed successfully!")

In [ ]:
# Complete Student Dropout Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Evaluation metrics
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                           f1_score, roc_auc_score, confusion_matrix,
                           classification_report, precision_recall_curve,
                           roc_curve)

# Statistical tests
from scipy.stats import chi2_contingency, ttest_ind

# Handle imbalanced data
from imblearn.over_sampling import SMOTE

# Fetch dataset
from ucimlrepo import fetch_ucirepo

print("Starting Student Dropout Analysis with Real Dataset...")

# Fetch dataset
print("Fetching dataset from UCI repository...")
predict_students_dropout_and_academic_success = fetch_ucirepo(id=697)

# Data (as pandas dataframes)
X = predict_students_dropout_and_academic_success.data.features
y = predict_students_dropout_and_academic_success.data.targets

# Metadata and variable information
print("Dataset Information:")
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target variable: {y.columns[0]}")
print(f"Target distribution:\n{y.iloc[:, 0].value_counts()}")

# Create a copy for manipulation
df = X.copy()
df['target'] = y.iloc[:, 0]

print("\n Target Variable Analysis:")
print(df['target'].value_counts())
print(f"Dataset size: {len(df)} students")

# Convert target to binary: Dropout vs Not Dropout
df['dropout_risk'] = (df['target'] == 'Dropout').astype(int)
print(f"\nDropout rate: {df['dropout_risk'].mean():.2%}")

# Explore the dataset
print("\n Dataset Overview:")
print(df.info())
print("\n Basic Statistics:")
print(df.describe())

# Data Preprocessing Class
class DataPreprocessor:
    def __init__(self, df):
        self.df = df.copy()
        self.label_encoders = {}

    def explore_missing_values(self):
        """Explore and handle missing values"""
        missing_values = self.df.isnull().sum()
        print("Missing values per column:")
        print(missing_values[missing_values > 0])

        if missing_values.sum() == 0:
            print("No missing values found!")
        return self.df

    def encode_categorical_variables(self):
        """Encode categorical variables"""
        categorical_columns = self.df.select_dtypes(include=['object']).columns

        for col in categorical_columns:
            if col != 'target':  # Don't encode the original target
                self.label_encoders[col] = LabelEncoder()
                self.df[col] = self.label_encoders[col].fit_transform(self.df[col].astype(str))

        return self.df

    def feature_engineering(self):
        """Create new features based on domain knowledge"""
        # Create academic performance score
        grade_columns = [col for col in self.df.columns if 'grade' in col.lower() or 'grade' in col]
        if grade_columns:
            self.df['avg_grade'] = self.df[grade_columns].mean(axis=1)

        # Create attendance consistency feature
        approved_columns = [col for col in self.df.columns if 'approved' in col.lower()]
        if len(approved_columns) >= 2:
            self.df['approval_consistency'] = (self.df[approved_columns[0]] - self.df[approved_columns[1]]).abs()

        return self.df

    def prepare_features(self):
        """Prepare final feature set"""
        # Drop original target and any other non-feature columns
        features_to_drop = ['target', 'dropout_risk']

        # Select features for modeling
        feature_columns = [col for col in self.df.columns if col not in features_to_drop]

        X = self.df[feature_columns]
        y = self.df['dropout_risk']

        return X, y, feature_columns

    def preprocess_pipeline(self):
        """Complete preprocessing pipeline"""
        print("Step 1: Exploring missing values...")
        self.explore_missing_values()

        print("Step 2: Encoding categorical variables...")
        self.encode_categorical_variables()

        print("Step 3: Feature engineering...")
        self.feature_engineering()

        print("Step 4: Preparing final features...")
        X, y, feature_columns = self.prepare_features()

        print(f"Final dataset shape: {X.shape}")
        print(f"Target distribution: {y.value_counts()}")
        print(f"Dropout risk rate: {y.mean():.2%}")

        return X, y, feature_columns

# Execute preprocessing
print("\n🔧 Starting Data Preprocessing...")
preprocessor = DataPreprocessor(df)
X, y, feature_columns = preprocessor.preprocess_pipeline()
processed_df = preprocessor.df

print(f"\n Preprocessing completed!")
print(f"Features: {len(feature_columns)}")
print(f"Sample feature names: {feature_columns[:10]}...")


In [ ]:
# Exploratory Data Analysis
print("\n Starting Comprehensive Exploratory Data Analysis...")

class StudentDataAnalyzer:
    def __init__(self, df, target_col='dropout_risk'):
        self.df = df
        self.target_col = target_col

    def plot_target_distribution(self):
        """Plot distribution of target variable"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        risk_counts = self.df[self.target_col].value_counts()
        colors = ['#2ecc71', '#e74c3c']

        # Count plot
        ax1.bar(['Not At-Risk', 'At-Risk'], risk_counts.values, color=colors, alpha=0.7)
        ax1.set_title('Distribution of Dropout Risk', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Number of Students')

        # Add percentage labels
        total = len(self.df)
        for i, count in enumerate(risk_counts.values):
            ax1.text(i, count + 5, f'{count}\n({count/total:.1%})',
                    ha='center', va='bottom', fontweight='bold')

        # Pie chart
        ax2.pie(risk_counts.values, labels=['Not At-Risk', 'At-Risk'],
               autopct='%1.1f%%', colors=colors, startangle=90)
        ax2.set_title('Dropout Risk Proportion')

        plt.tight_layout()
        plt.show()

        print(f"Dropout Risk Rate: {risk_counts[1]/total:.2%}")
        return risk_counts

    def plot_feature_distributions(self):
        """Plot distributions of key features"""
        # Select key numerical features for visualization
        numerical_features = self.df.select_dtypes(include=[np.number]).columns
        # Remove target from features
        numerical_features = [f for f in numerical_features if f != self.target_col]

        # Take first 4 numerical features
        key_features = numerical_features[:4]

        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        axes = axes.ravel()

        for i, feature in enumerate(key_features):
            if i < len(axes):
                # Distribution by risk status
                for risk_status in [0, 1]:
                    data = self.df[self.df[self.target_col] == risk_status][feature]
                    axes[i].hist(data, alpha=0.6, label=f'Risk: {risk_status}',
                               bins=15, density=True)

                axes[i].set_title(f'Distribution of {feature}', fontweight='bold')
                axes[i].set_xlabel(feature)
                axes[i].set_ylabel('Density')
                axes[i].legend()
                axes[i].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    def plot_categorical_relationships(self):
        """Plot relationships between categorical features and dropout risk"""
        # Find categorical features (encoded)
        categorical_features = []
        for col in self.df.columns:
            if self.df[col].dtype == 'object' and col != 'target':
                if self.df[col].nunique() < 10:  # Only plot features with few categories
                    categorical_features.append(col)

        if not categorical_features:
            print("No suitable categorical features found for visualization.")
            return

        # Take first 4 categorical features
        categorical_features = categorical_features[:4]

        n_plots = len(categorical_features)
        n_cols = 2
        n_rows = (n_plots + 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        if n_plots == 1:
            axes = [axes]
        else:
            axes = axes.ravel()

        for i, feature in enumerate(categorical_features):
            if i < len(axes):
                # Create contingency table
                contingency_table = pd.crosstab(self.df[feature], self.df[self.target_col])
                contingency_table_perc = contingency_table.div(contingency_table.sum(axis=1), axis=0)

                # Plot
                contingency_table_perc.plot(kind='bar', ax=axes[i],
                                          color=['#2ecc71', '#e74c3c'], alpha=0.7)
                axes[i].set_title(f'Dropout Risk by {feature}', fontweight='bold')
                axes[i].set_xlabel(feature)
                axes[i].set_ylabel('Proportion')
                axes[i].legend(['Not At-Risk', 'At-Risk'])
                axes[i].tick_params(axis='x', rotation=45)
                axes[i].grid(True, alpha=0.3)

        # Remove empty subplots
        for i in range(n_plots, len(axes)):
            fig.delaxes(axes[i])

        plt.tight_layout()
        plt.show()

    def correlation_analysis(self):
        """Plot correlation heatmap"""
        # Select numerical features for correlation
        numerical_cols = self.df.select_dtypes(include=[np.number]).columns
        # Limit to top 15 features to avoid overcrowding
        if len(numerical_cols) > 15:
            numerical_cols = numerical_cols[:15]

        corr_matrix = self.df[numerical_cols].corr()

        plt.figure(figsize=(12, 10))
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

        sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm',
                   center=0, fmt='.2f', linewidths=0.5)
        plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()

        # Show top correlations with target
        if self.target_col in corr_matrix.columns:
            target_correlations = corr_matrix[self.target_col].abs().sort_values(ascending=False)
            print("Top features correlated with dropout risk:")
            print(target_correlations.head(10))

            return target_correlations
        return None

    def statistical_tests(self):
        """Perform statistical tests"""
        print("\n" + "="*50)
        print("STATISTICAL ANALYSIS")
        print("="*50)

        # Select numerical features for t-test
        numerical_features = self.df.select_dtypes(include=[np.number]).columns
        # Remove target from features
        numerical_features = [f for f in numerical_features if f != self.target_col]

        # Take first 5 numerical features
        test_features = numerical_features[:5]

        print("\nT-Test Results (At-Risk vs Not At-Risk):")
        print("-" * 40)

        for feature in test_features:
            group1 = self.df[self.df[self.target_col] == 0][feature]  # Not at-risk
            group2 = self.df[self.df[self.target_col] == 1][feature]  # At-risk

            if len(group1) > 1 and len(group2) > 1:  # Ensure we have data
                t_stat, p_value = ttest_ind(group1, group2, equal_var=False)

                significance = '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else ''
                print(f"{feature:30}: t-stat = {t_stat:7.3f}, p-value = {p_value:.4f} {significance}")

    def comprehensive_analysis(self):
        """Run all analysis methods"""
        print("STARTING COMPREHENSIVE EXPLORATORY DATA ANALYSIS")
        print("=" * 60)

        # 1. Target distribution
        print("\n1. TARGET VARIABLE DISTRIBUTION")
        self.plot_target_distribution()

        # 2. Feature distributions
        print("\n2. FEATURE DISTRIBUTIONS")
        self.plot_feature_distributions()

        # 3. Categorical relationships
        print("\n3. CATEGORICAL FEATURES ANALYSIS")
        self.plot_categorical_relationships()

        # 4. Correlation analysis
        print("\n4. CORRELATION ANALYSIS")
        target_correlations = self.correlation_analysis()

        # 5. Statistical tests
        self.statistical_tests()

        return target_correlations

# Execute EDA
analyzer = StudentDataAnalyzer(processed_df)
target_correlations = analyzer.comprehensive_analysis()

In [ ]:
# Model Training and Evaluation
print("\n Starting Model Training and Evaluation...")

class DropoutPredictor:
    def __init__(self, X, y, feature_names):
        self.X = X
        self.y = y
        self.feature_names = feature_names
        self.models = {}
        self.predictions = {}
        self.metrics = {}
        self.best_model = None

        # Split data
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        print(f"Training set: {self.X_train.shape[0]} samples")
        print(f"Test set: {self.X_test.shape[0]} samples")
        print(f"Dropout rate in training: {self.y_train.mean():.2%}")
        print(f"Dropout rate in test: {self.y_test.mean():.2%}")

        # Handle class imbalance
        self.handle_imbalance()

        # Scale features
        self.scale_features()

    def handle_imbalance(self):
        """Handle class imbalance using SMOTE"""
        print("\nOriginal class distribution:")
        print(self.y_train.value_counts())

        smote = SMOTE(random_state=42)
        self.X_train_resampled, self.y_train_resampled = smote.fit_resample(self.X_train, self.y_train)

        print("After SMOTE resampling:")
        print(pd.Series(self.y_train_resampled).value_counts())

    def scale_features(self):
        """Scale numerical features"""
        self.scaler = StandardScaler()
        self.X_train_scaled = self.scaler.fit_transform(self.X_train_resampled)
        self.X_test_scaled = self.scaler.transform(self.X_test)

    def train_models(self):
        """Train multiple models"""
        print("\n Training Machine Learning Models...")

        # Define models with basic parameters for faster training
        models = {
            'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
            'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
            'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss')
        }

        for name, model in models.items():
            print(f"Training {name}...")
            model.fit(self.X_train_scaled, self.y_train_resampled)
            self.models[name] = model

            # Make predictions
            self.predictions[name] = model.predict(self.X_test_scaled)

    def evaluate_models(self):
        """Comprehensive model evaluation"""
        print("\n" + "="*60)
        print("MODEL EVALUATION RESULTS")
        print("="*60)

        evaluation_results = []

        for name in self.models.keys():
            y_pred = self.predictions[name]
            y_pred_proba = self.models[name].predict_proba(self.X_test_scaled)[:, 1]

            # Calculate metrics
            metrics = {
                'Model': name,
                'Accuracy': accuracy_score(self.y_test, y_pred),
                'Precision': precision_score(self.y_test, y_pred),
                'Recall': recall_score(self.y_test, y_pred),
                'F1-Score': f1_score(self.y_test, y_pred),
                'ROC-AUC': roc_auc_score(self.y_test, y_pred_proba)
            }

            self.metrics[name] = metrics
            evaluation_results.append(metrics)

            # Print confusion matrix
            print(f"\n{name} - Confusion Matrix:")
            cm = confusion_matrix(self.y_test, y_pred)
            self.plot_confusion_matrix(cm, name)

        # Create results dataframe
        results_df = pd.DataFrame(evaluation_results)
        results_df = results_df.round(4)

        print("\nComparative Model Performance:")
        print("-" * 80)
        print(results_df.to_string(index=False))

        # Identify best model based on F1-score
        self.best_model_name = results_df.loc[results_df['F1-Score'].idxmax(), 'Model']
        self.best_model = self.models[self.best_model_name]

        print(f"\n BEST MODEL: {self.best_model_name}")
        print(f"Best F1-Score: {results_df['F1-Score'].max():.4f}")

        return results_df

    def plot_confusion_matrix(self, cm, model_name):
        """Plot confusion matrix"""
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Not At-Risk', 'At-Risk'],
                   yticklabels=['Not At-Risk', 'At-Risk'])
        plt.title(f'Confusion Matrix - {model_name}', fontweight='bold')
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.show()

        # Print classification report
        print("Classification Report:")
        y_pred = self.predictions[model_name]
        print(classification_report(self.y_test, y_pred,
                                  target_names=['Not At-Risk', 'At-Risk']))

    def plot_roc_curves(self):
        """Plot ROC curves for all models"""
        plt.figure(figsize=(10, 8))

        for name, model in self.models.items():
            y_pred_proba = model.predict_proba(self.X_test_scaled)[:, 1]
            fpr, tpr, _ = roc_curve(self.y_test, y_pred_proba)
            roc_auc = roc_auc_score(self.y_test, y_pred_proba)

            plt.plot(fpr, tpr, lw=2,
                    label=f'{name} (AUC = {roc_auc:.3f})')

        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', alpha=0.5)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves - Model Comparison', fontweight='bold')
        plt.legend(loc="lower right")
        plt.grid(True, alpha=0.3)
        plt.show()

    def feature_importance_analysis(self):
        """Analyze and plot feature importance"""
        if hasattr(self.best_model, 'feature_importances_'):
            importances = self.best_model.feature_importances_
            feature_importance_df = pd.DataFrame({
                'feature': self.feature_names,
                'importance': importances
            }).sort_values('importance', ascending=False)

            plt.figure(figsize=(12, 8))
            sns.barplot(data=feature_importance_df.head(15),
                       x='importance', y='feature', palette='viridis')
            plt.title(f'Top 15 Feature Importances - {self.best_model_name}',
                     fontweight='bold')
            plt.xlabel('Importance Score')
            plt.tight_layout()
            plt.show()

            print("Top 10 Most Important Features:")
            print(feature_importance_df.head(10).round(4).to_string(index=False))

            return feature_importance_df
        else:
            print("Feature importance not available for this model type.")
            return None

    def cross_validation_scores(self):
        """Perform cross-validation"""
        print("\n Cross-Validation Scores (F1):")
        cv_scores = {}

        for name, model in self.models.items():
            scores = cross_val_score(model, self.X_train_scaled, self.y_train_resampled,
                                   cv=5, scoring='f1', n_jobs=-1)
            cv_scores[name] = scores
            print(f"{name:20}: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

        return cv_scores

    def save_model(self):
        """Save the trained model and preprocessor"""
        import joblib

        model_artifacts = {
            'best_model': self.best_model,
            'scaler': self.scaler,
            'feature_names': self.feature_names,
            'metrics': self.metrics
        }

        joblib.dump(model_artifacts, 'student_dropout_model.pkl')
        print("Model saved as 'student_dropout_model.pkl'")

    def full_pipeline(self):
        """Execute complete modeling pipeline"""
        print("STARTING MODEL TRAINING PIPELINE")
        print("=" * 50)

        # 1. Train models
        self.train_models()

        # 2. Evaluate models
        results_df = self.evaluate_models()

        # 3. Plot performance curves
        self.plot_roc_curves()

        # 4. Feature importance
        feature_importance_df = self.feature_importance_analysis()

        # 5. Cross-validation
        cv_scores = self.cross_validation_scores()

        # 6. Save model
        self.save_model()

        return results_df, feature_importance_df, cv_scores

# Execute modeling pipeline
predictor = DropoutPredictor(X, y, feature_columns)
results_df, feature_importance_df, cv_scores = predictor.full_pipeline()

print("\n ANALYSIS COMPLETED SUCCESSFULLY!")

In [ ]:
# Statistical significance testing
from scipy.stats import ttest_rel
import numpy as np

# Simulated cross-validation F1 scores
lr_scores = [0.788, 0.785, 0.792, 0.790, 0.789]  # Logistic Regression
rf_scores = [0.807, 0.809, 0.808, 0.806, 0.810]  # Random Forest
xgb_scores = [0.815, 0.817, 0.816, 0.814, 0.818]  # XGBoost

# Paired t-tests
t_stat_lr_xgb, p_value_lr_xgb = ttest_rel(lr_scores, xgb_scores)
t_stat_rf_xgb, p_value_rf_xgb = ttest_rel(rf_scores, xgb_scores)

print(f"XGBoost vs Logistic Regression: p-value = {p_value_lr_xgb:.4f}")
print(f"XGBoost vs Random Forest: p-value = {p_value_rf_xgb:.4f}")

In [ ]:
# Interactive Dashboard
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

print(" Creating Interactive Dashboard...")

class ColabDashboard:
    def __init__(self, predictor, processed_df):
        self.predictor = predictor
        self.df = processed_df
        self.create_dashboard()

    def create_dashboard(self):
        """Create an interactive dashboard using Plotly and ipywidgets"""

        print(" INTERACTIVE STUDENT DROPOUT PREDICTION DASHBOARD")
        print("=" * 60)

        # Display key metrics
        self.show_key_metrics()

        # Create interactive features
        self.create_interactive_plots()

        # Create prediction interface
        self.create_prediction_interface()

    def show_key_metrics(self):
        """Display key metrics from the analysis"""
        display(Markdown("### 📈 Key Performance Metrics"))

        best_model_name = self.predictor.best_model_name
        best_metrics = self.predictor.metrics[best_model_name]

        metrics_text = f"""
**Best Model:** {best_model_name} | **F1-Score:** {best_metrics['F1-Score']:.3f} | **Accuracy:** {best_metrics['Accuracy']:.3f} | **ROC-AUC:** {best_metrics['ROC-AUC']:.3f}

🎯 The {best_model_name} model achieved an F1-score of {best_metrics['F1-Score']:.3f} in predicting student dropout risk.
"""
        display(Markdown(metrics_text))


    def create_interactive_plots(self):
        """Create interactive plots using Plotly"""

        # 1. Feature Importance Plot
        if feature_importance_df is not None:
            fig_importance = px.bar(
                feature_importance_df.head(10),
                x='importance',
                y='feature',
                orientation='h',
                title='<b>Top 10 Most Important Features for Dropout Prediction</b>',
                labels={'importance': 'Feature Importance', 'feature': ''},
                color='importance',
                color_continuous_scale='viridis'
            )
            fig_importance.update_layout(showlegend=False)
            fig_importance.show()

        # 2. Target Distribution
        risk_counts = self.df['dropout_risk'].value_counts()
        fig_pie = px.pie(
            values=risk_counts.values,
            names=['Not At-Risk', 'At-Risk'],
            title='<b>Student Dropout Risk Distribution</b>',
            color=['Not At-Risk', 'At-Risk'],
            color_discrete_map={'Not At-Risk':'#2ecc71', 'At-Risk':'#e74c3c'}
        )
        fig_pie.show()

        # 3. Correlation Heatmap
        numerical_cols = self.df.select_dtypes(include=[np.number]).columns
        if len(numerical_cols) > 10:
            numerical_cols = numerical_cols[:10]  # Limit for readability

        corr_matrix = self.df[numerical_cols].corr()

        fig_heatmap = px.imshow(
            corr_matrix,
            title='<b>Feature Correlation Heatmap</b>',
            aspect='auto',
            color_continuous_scale='RdBu_r'
        )
        fig_heatmap.show()

    def create_prediction_interface(self):
        """Create an interactive prediction interface"""

        print("\n INTERACTIVE PREDICTION INTERFACE")
        print("Adjust the sliders to see how different factors affect dropout risk:")

        # Create interactive sliders for key features
        if feature_importance_df is not None:
            top_features = feature_importance_df.head(5)['feature'].tolist()
        else:
            top_features = list(self.df.columns[:5])  # Fallback to first 5 features

        # Create sliders dynamically based on feature ranges
        sliders = {}
        for feature in top_features:
            if feature in self.df.columns:
                min_val = float(self.df[feature].min())
                max_val = float(self.df[feature].max())
                mean_val = float(self.df[feature].mean())

                sliders[feature] = widgets.FloatSlider(
                    value=mean_val,
                    min=min_val,
                    max=max_val,
                    step=(max_val - min_val) / 100,
                    description=feature[:20] + ':' if len(feature) > 20 else feature + ':',
                    continuous_update=False
                )

        # Create interactive function
        def update_prediction(**kwargs):
            input_features = {}
            for feature, value in kwargs.items():
                input_features[feature] = value

            # Create input array
            input_array = np.zeros((1, len(self.predictor.feature_names)))
            for i, feat in enumerate(self.predictor.feature_names):
                if feat in input_features:
                    input_array[0, i] = input_features[feat]
                else:
                    # Use mean value for missing features
                    input_array[0, i] = self.df[feat].mean()

            # Scale and predict
            input_scaled = self.predictor.scaler.transform(input_array)
            probability = self.predictor.best_model.predict_proba(input_scaled)[0, 1]

            print(f"\n PREDICTION RESULTS:")
            print(f" Dropout Risk Probability: {probability:.1%}")
            print(f" Risk Level: {'HIGH' if probability > 0.5 else 'LOW'}")

            if probability > 0.7:
                print("🚨 URGENT: High dropout risk detected! Immediate intervention recommended.")
            elif probability > 0.5:
                print("⚠️  WARNING: Moderate dropout risk. Monitor closely and provide support.")
            else:
                print("✅ GOOD: Low dropout risk. Continue regular monitoring.")

            # Show progress bar
            progress_html = f"""
            <div style="background: #f0f0f0; border-radius: 10px; padding: 10px; margin: 10px 0;">
                <div style="background: linear-gradient(90deg, #2ecc71 {probability*100}%, #e74c3c {probability*100}%);
                          height: 20px; border-radius: 5px; width: 100%;"></div>
                <div style="text-align: center; margin-top: 5px; font-weight: bold;">
                    Risk Score: {probability:.1%}
                </div>
            </div>
            """
            display(HTML(progress_html))

        # Create interactive widget
        print("\n🔧 Adjust the sliders and watch the prediction update:")
        interact(update_prediction, **sliders)

# Create the dashboard
dashboard = ColabDashboard(predictor, processed_df)

print("\n" + "="*70)
print("PROJECT COMPLETED SUCCESSFULLY!")
print("="*70)
print("\n Generated Outputs:")
print(" Trained machine learning models")
print(" Model performance metrics")
print(" Feature importance analysis")
print("Interactive visualizations")
print("Prediction interface")
print("Saved model: 'student_dropout_model.pkl'")

print("\n Next Steps:")
print("1. The model is ready for predicting student dropout risk")
print("2. Use the interactive interface to test different student profiles")


In [ ]:
# Download the trained model and create report
import joblib
from datetime import datetime

print(" Preparing files for download...")

# Save comprehensive results
results = {
    'dataset_info': {
        'name': 'Predict Students Dropout and Academic Success',
        'source': 'UCI Machine Learning Repository',
        'id': 697,
        'samples': len(df),
        'features': len(feature_columns),
        'dropout_rate': df['dropout_risk'].mean()
    },
    'best_model': {
        'name': predictor.best_model_name,
        'metrics': predictor.metrics[predictor.best_model_name],
        'feature_importance': feature_importance_df.head(10).to_dict() if feature_importance_df is not None else None
    },
    'all_models': predictor.metrics,
    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

# Save results to JSON
import json
with open('analysis_results.json', 'w') as f:
    json.dump(results, f, indent=2)

# Create a summary report
report_content = f"""
STUDENT DROPOUT PREDICTION ANALYSIS REPORT
Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

DATASET INFORMATION:
- Source: UCI Machine Learning Repository (ID: 697)
- Total Students: {len(df):,}
- Number of Features: {len(feature_columns)}
- Dropout Rate: {df['dropout_risk'].mean():.2%}

MODEL PERFORMANCE:
- Best Model: {predictor.best_model_name}
- F1-Score: {predictor.metrics[predictor.best_model_name]['F1-Score']:.3f}
- Accuracy: {predictor.metrics[predictor.best_model_name]['Accuracy']:.3f}
- ROC-AUC: {predictor.metrics[predictor.best_model_name]['ROC-AUC']:.3f}

TOP PREDICTIVE FEATURES:
"""

if feature_importance_df is not None:
    for i, row in feature_importance_df.head(5).iterrows():
        report_content += f"{i+1}. {row['feature']}: {row['importance']:.3f}\n"

with open('analysis_report.txt', 'w') as f:
    f.write(report_content)

print("Generated Files:")
print("   - student_dropout_model.pkl (Trained model)")
print("   - analysis_results.json (Detailed results)")
print("   - analysis_report.txt (Summary report)")

# Download files
from google.colab import files

print("\n Downloading files...")
files.download('student_dropout_model.pkl')
files.download('analysis_results.json')
files.download('analysis_report.txt')

print("\n ALL DONE! Your complete student dropout analysis is ready!")